In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import pandas as pd

c:\Users\sairam\AppData\Local\Programs\Python\Python311\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# ====== CONFIGURATION ======
KNOWN_FACES_PATH = r"C:\Users\sairam\Videos\data set\face attendance\known faces"
ATTENDANCE_FILE = r"C:\Users\sairam\Videos\data set\face attendance\Office_Attendance_Master.csv"

In [3]:
# ====== LOAD KNOWN FACES ======
images = []
names = []
for file in os.listdir(KNOWN_FACES_PATH):
    img = cv2.imread(f'{KNOWN_FACES_PATH}/{file}')
    images.append(img)
    names.append(os.path.splitext(file)[0])  # filename without extension

In [4]:
# ====== ENCODING ======
def find_encodings(images):
    encode_list = []
    for img in images:
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(rgb_img)[0]
        encode_list.append(encode)
    return encode_list

known_encodings = find_encodings(images)
print('✅ All employee faces encoded successfully!')

✅ All employee faces encoded successfully!


In [5]:
# ====== MARK ATTENDANCE ======
def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    time_now = now.strftime('%H:%M:%S')
    hour, minute = now.hour, now.minute

    # Create file if not exists
    if not os.path.exists(ATTENDANCE_FILE):
        pd.DataFrame(columns=['Name', 'Date', 'Time', 'Status', 'Punctuality']).to_csv(ATTENDANCE_FILE, index=False)

    df = pd.read_csv(ATTENDANCE_FILE)

    # Check if already marked for today
    already_marked = ((df['Name'] == name) & (df['Date'] == date_today)).any()

    if not already_marked:
        # Office hours: 9 AM to 6 PM
        if hour < 9:
            status = "Present"
            punctuality = "On Time"
        elif 9 <= hour < 18:
            status = "Present"
            # Late if after 9:00:00
            if hour == 9 and minute == 0:
                punctuality = "On Time"
            else:
                punctuality = "Late"
        else:
            status = "After Hours"
            punctuality = "Late"

        new_entry = pd.DataFrame([[name, date_today, time_now, status, punctuality]],
                                 columns=['Name', 'Date', 'Time', 'Status', 'Punctuality'])
        df = pd.concat([df, new_entry], ignore_index=True)
        df.to_csv(ATTENDANCE_FILE, index=False)
        print(f"🟢 Attendance marked for: {name} ({punctuality}) at {time_now}")
    else:
        print(f"ℹ️ {name} already marked today.")

In [6]:
# ====== START WEBCAM ======
cap = cv2.VideoCapture(0)
print("🎥 Camera started. Press 'Q' to quit.")

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture frame.")
        break

    small_img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    rgb_small_img = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)

    faces_cur_frame = face_recognition.face_locations(rgb_small_img)
    encodes_cur_frame = face_recognition.face_encodings(rgb_small_img, faces_cur_frame)

    for encodeFace, faceLoc in zip(encodes_cur_frame, faces_cur_frame):
        matches = face_recognition.compare_faces(known_encodings, encodeFace)
        face_dis = face_recognition.face_distance(known_encodings, encodeFace)
        matchIndex = np.argmin(face_dis)

        if matches[matchIndex]:
            name = names[matchIndex].upper()
            color = (0, 255, 0)
            mark_attendance(name)
        else:
            name = 'UNKNOWN'
            color = (0, 0, 255)

        y1, x2, y2, x1 = [v * 4 for v in faceLoc]
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow('Smart Office Entry Camera', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("🛑 Attendance system stopped.")


🎥 Camera started. Press 'Q' to quit.
🟢 Attendance marked for: SAIRAM (Late) at 15:51:10
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
ℹ️ SAIRAM already marked today.
🛑 Attendance system stopped.
